In [19]:
import numpy as np

# Normalize the data.
from sklearn import preprocessing
from numpy.linalg import cholesky, det, lstsq
from scipy.optimize import minimize
import scipy.spatial.distance as spdist

def pass_arg_loss(Xx, nsim, tr_size):

    #     Compute the RMSE
    def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean((y_pred-y_true)**2))
    
    print("tr_Size:",tr_size)
    
    # Compute the RMSE
    # def root_mean_squared_error(y_true, y_pred):
        # return np.sqrt(np.mean((y_pred-y_true)**2))
    
    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        porofn = -porof*(porof<0)
        porofp = porof*(porof>=poroi) - poroi*(porof>=poroi)
        return porofp+porofn
    
    # Experimental data
    data = np.loadtxt('../data/labeled_data.dat')
    x_labeled = data[:, :2].astype(np.float64) # -2 because we do not need porosity predictions
    y_labeled = data[:, -2:-1].astype(np.float64) # dimensionless bond length and porosity measurements

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_labeled = scaler.fit_transform(x_labeled)
    # y_labeled = scaler.fit_transform(y_labeled)

    tr_size = int(tr_size)

    # train and test data
    trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]

    # Physics data
    data_phyloss = np.loadtxt('../data/unlabeled_data_BK_constw_v2_1525.dat')
    x_unlabeled = data_phyloss[:, :]
    initporo = x_unlabeled[:, -1]

    x_unlabeled1 = x_unlabeled[:1303, :2]
    x_unlabeled2 = x_unlabeled[-6:, :2]
    y_unlabeled1 = data_phyloss[:1303, -2:-1]
    y_unlabeled2 = data_phyloss[-6:, -2:-1]

    x_unlabeled = np.vstack((x_unlabeled1,x_unlabeled2))
    y_unlabeled = np.vstack((y_unlabeled1,y_unlabeled2))

    # normalize dataset with MinMaxScaler
    scaler = preprocessing.MinMaxScaler(feature_range=(0.0, 1.0))
    x_unlabeled = scaler.fit_transform(x_unlabeled)

    # initial porosity
    init_poro1 = initporo[:1303]
    init_poro2 = initporo[-6:]
    init_poro = np.hstack((init_poro1,init_poro2))


    def covSEard(hyp=None, x=None, z=None):
        ''' Squared Exponential covariance function with Automatic Relevance Detemination
         (ARD) distance measure. The covariance function is parameterized as:

         k(x^p,x^q) = sf2 * exp(-(x^p - x^q)' * inv(P) * (x^p - x^q)/2)

         where the P matrix is diagonal with ARD parameters ell_1^2,...,ell_D^2, where
         D is the dimension of the input space and sf2 is the signal variance.

         The hyperparameters are:

         hyp = [ log(ell_1)
                 log(ell_2)
                 ...
                 log(ell_D)
                 log(sqrt(sf2)) ]
        '''

        [n, D] = x.shape
        ell = 1/np.array(hyp[0:D])        # characteristic length scale
        
        
        sf2 = np.array(hyp[D])**2         # signal variance
        tmp = np.dot(np.diag(ell),x.T).T
        A = spdist.cdist(np.dot(np.diag(ell),x.T).T, np.dot(np.diag(ell),z.T).T, 'sqeuclidean') # cross covariances
        A = sf2*np.exp(-0.5*A)  

        return A


    def posterior_predictive(X_s, X_train, Y_train, l1=.1, l2=.1, sigma_f=.1, sigma_y=0):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''
        K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_train) + sigma_y**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z=X_s)  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, cov_s


    def nll_fn(X_train, Y_train, noise=0, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''

        def nll_stable(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = covSEard(hyp=[theta[0],theta[1],theta[2]], x=X_train, z=X_train) + \
                noise**2 * np.eye(len(X_train))
            
            K += 1e-6 * np.eye(*K.shape)
            L = cholesky(K)
            return np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)

        if naive:
            return nll_naive
        else:
            return nll_stable

        
    def posterior_predictive_discp(X_s, X_train, Y_train, l1=.1, l2=.1, sigma_f=.1, sigma_y=1e-2):
        '''  
        Computes the suffifient statistics of the GP posterior predictive distribution 
        from m training data X_train and Y_train and n new inputs X_s.

        Args:
            X_s: New input locations (n x d).
            X_train: Training locations (m x d).
            Y_train: Training targets (m x 1).
            l: Kernel length parameter.
            sigma_f: Kernel vertical variation parameter.
            sigma_y: Noise parameter.

        Returns:
            Posterior mean vector (n x d) and covariance matrix (n x n).
        '''
        K = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_train) + sigma_y**2 * np.eye(len(X_train))
        K_s = covSEard(hyp=[l1,l2,sigma_f], x=X_train, z=X_s)
        K_ss = covSEard(hyp=[l1,l2,sigma_f], x=X_s, z=X_s)  + 1e-8 * np.eye(len(X_s))
#         K_inv = inv(K)
        K_inv = np.linalg.pinv(K)
    
        # Equation (4)
        mu_s = K_s.T.dot(K_inv).dot(Y_train)

        # Equation (5)
        cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)
        
        return mu_s, cov_s


    def nll_fn_discp(X_train, Y_train, x_unlabeled, init_poro, phy_pred, naive=False):
        '''
        Returns a function that computes the negative log marginal
        likelihood for training data X_train and Y_train and given 
        noise level.

        Args:
            X_train: training locations (m x d).
            Y_train: training targets (m x 1).
            noise: known noise level of Y_train.
            naive: if True use a naive implementation of Eq. (7), if 
                   False use a numerically more stable implementation. 

        Returns:
            Minimization objective.
        '''

        def nll_stable_discp(theta):
            # Numerically more stable implementation of Eq. (7) as described
            # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
            # 2.2, Algorithm 2.1.
            K = covSEard(hyp=[theta[0],theta[1],theta[2]], x=X_train, z=X_train) + \
                theta[3]**2 * np.eye(len(X_train))
            
            K += 1e-6 * np.eye(*K.shape)
            L = cholesky(K)
#             return np.sum(np.log(np.diagonal(L))) + \
#                    0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
#                    0.5 * len(X_train) * np.log(2*np.pi)
        
            mu_discp, _ = posterior_predictive_discp(x_unlabeled, X_train, Y_train, l1=theta[0], l2=theta[1], sigma_f=theta[2], sigma_y=theta[3])
            
            # physics predictions GP1+GP2 (for physics inputs)
            mu_un = phy_pred + mu_discp
            phyloss_poro = np.mean(poros(init_poro, mu_un))

            log_loss = np.sum(np.log(np.diagonal(L))) + \
                   0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
                   0.5 * len(X_train) * np.log(2*np.pi)
        
            print(5e5*phyloss_poro,log_loss, theta, 5e7*phyloss_poro)
            return 5e7*phyloss_poro + log_loss
        
        

        return nll_stable_discp

        
    
    # Optimization - GP1 - for physics
    res1 = minimize(nll_fn(x_unlabeled, y_unlabeled), x0 = [.1, .1, .1], 
                   bounds=((1e-5, None), (1e-5, None), (1e-5, None)),
                    method='L-BFGS-B')
    
    # GP (physics) predictions for the experimental sets
    mu_s1, cov_s1 = posterior_predictive(trainX, x_unlabeled, y_unlabeled, *res1.x)
    
    # GP (physics) predictions for the physics sets
    mu_phy, cov_phy = posterior_predictive(x_unlabeled, x_unlabeled, y_unlabeled, *res1.x)
    
    # discrepancy
    discp = trainY - mu_s1
    
    # Optimization - GP2 - for discrepency
    res2 = minimize(nll_fn_discp(trainX, discp, x_unlabeled, init_poro, mu_phy), x0 = [.1, .1, .1, 1e-3], 
                   bounds=((1e-5, None), (1e-5, None), (1e-5, None), (1e-7, None)),
                    method='L-BFGS-B')
    
    # GP (physics) predictions
    # mu_s2, cov_s2 = posterior_predictive_discp(trainX, trainX, discp, *res2.x)
    
    # GP (physics) predictions
    mu_s3, cov_s3 = posterior_predictive(testX, x_unlabeled, y_unlabeled, *res1.x)
    
    # GP (physics) predictions
    mu_s4, cov_s4 = posterior_predictive_discp(testX, trainX, discp, *res2.x)
    
#     print(mu_s2+mu_s1, discp)
    pred_mu = mu_s3+mu_s4
    pred_cov = cov_s3+cov_s4
    
    
    
    # pred_mu = mu_s2+mu_s1
    # pred_cov = cov_s2+cov_s1
    # samples = np.random.multivariate_normal(pred_mu.ravel(), pred_cov, int(nsim))
    
#     print(f'After parameter optimization: l1={res.x[0]:.5f} l2={res.x[1]:.5f} sigma_f={res.x[2]:.5f}')
#     print(np.exp(res.x[0]),np.exp(res.x[1]), np.exp(res.x[2]))
    
    RMSE = []
    for ii in range(int(nsim)):
        samples = np.random.multivariate_normal(pred_mu.ravel(), pred_cov, 1)
        RMSE.append(root_mean_squared_error(testY, samples))
        print("RMSE:", root_mean_squared_error(testY, samples))
        

    return samples, RMSE

In [20]:
Xx = np.random.uniform(size=(1, 2))
ss, rmse = pass_arg_loss(Xx, 100, 30)

tr_Size: 30


C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:154: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:231: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


0.7248750121721971 [[53.15844473]] [0.1   0.1   0.1   0.001] 72.4875012172197
0.7248738588819653 [[53.15844426]] [0.10000001 0.1        0.1        0.001     ] 72.48738588819653
0.7248734749150747 [[53.15844397]] [0.1        0.10000001 0.1        0.001     ] 72.48734749150748
0.7248750129694329 [[53.15844464]] [0.1        0.1        0.10000001 0.001     ] 72.4875012969433
0.7248749356698158 [[53.15715348]] [0.1        0.1        0.1        0.00100001] 72.48749356698157
0.0 [[27.6407769]] [0.18818428 0.21764925 0.10001015 0.99013204] 0.0
0.0 [[27.6407769]] [0.18818429 0.21764925 0.10001015 0.99013204] 0.0
0.0 [[27.6407769]] [0.18818428 0.21764926 0.10001015 0.99013204] 0.0
0.0 [[27.64077692]] [0.18818428 0.21764925 0.10001016 0.99013204] 0.0
0.0 [[27.64077719]] [0.18818428 0.21764925 0.10001015 0.99013205] 0.0
0.0 [[-2.849489]] [0.12939782 0.1392205  0.10000338 0.33074501] 0.0
0.0 [[-2.84948902]] [0.12939783 0.1392205  0.10000338 0.33074501] 0.0
0.0 [[-2.84948902]] [0.12939782 0.13922051

0.0 [[-49.97395602]] [0.14053029 0.15052672 0.06309064 0.02313142] 0.0
0.0 [[-49.97395595]] [0.14053028 0.15052673 0.06309064 0.02313142] 0.0
0.0 [[-49.97395776]] [0.14053028 0.15052672 0.06309065 0.02313142] 0.0
0.0 [[-49.9739495]] [0.14053028 0.15052672 0.06309064 0.02313143] 0.0
317.40676988154587 [[176.7628025]] [1.63185271e-01 1.73208569e-01 1.26745184e-01 1.00000000e-07] 31740.676988154588
317.9908547624836 [[176.76280201]] [1.63185281e-01 1.73208569e-01 1.26745184e-01 1.00000000e-07] 31799.085476248358
318.0220879795266 [[176.76280177]] [1.63185271e-01 1.73208579e-01 1.26745184e-01 1.00000000e-07] 31802.20879795266
317.1450261154729 [[176.76280335]] [1.63185271e-01 1.73208569e-01 1.26745194e-01 1.00000000e-07] 31714.502611547287
317.75150252637275 [[176.76280195]] [1.63185271e-01 1.73208569e-01 1.26745184e-01 1.10000000e-07] 31775.150252637275
64.23263445773904 [[-58.58949324]] [0.14807836 0.15808375 0.08429874 0.01542464] 6423.263445773903
64.23265865925181 [[-58.58949387]] [0.

0.07248453399800152 [[-52.98572881]] [0.14284678 0.15288897 0.06900944 0.02075418] 7.248453399800151
0.07248530745983892 [[-52.98572959]] [0.14284679 0.15288897 0.06900944 0.02075418] 7.248530745983892
0.0724853323545653 [[-52.98572954]] [0.14284678 0.15288898 0.06900944 0.02075418] 7.24853323545653
0.07248542889878347 [[-52.98573043]] [0.14284678 0.15288897 0.06900945 0.02075418] 7.248542889878347
0.07248155837397839 [[-52.98572257]] [0.14284678 0.15288897 0.06900944 0.02075419] 7.248155837397839
0.0 [[-52.53474157]] [0.14245482 0.15247663 0.06818142 0.02115995] 0.0
0.0 [[-52.53474236]] [0.14245483 0.15247663 0.06818142 0.02115995] 0.0
0.0 [[-52.53474231]] [0.14245482 0.15247664 0.06818142 0.02115995] 0.0
0.0 [[-52.53474331]] [0.14245482 0.15247663 0.06818143 0.02115995] 0.0
0.0 [[-52.53473542]] [0.14245482 0.15247663 0.06818142 0.02115996] 0.0
0.0 [[-52.83313463]] [0.14271351 0.15274877 0.06872792 0.02089214] 0.0
0.0 [[-52.83313541]] [0.14271352 0.15274877 0.06872792 0.02089214] 0.0


0.5716505946372543 [[-53.27129597]] [0.14327936 0.15335459 0.06981551 0.02060608] 57.165059463725434
0.5716509759916542 [[-53.27129675]] [0.14327936 0.15335458 0.06981552 0.02060608] 57.16509759916542
0.5716363828166282 [[-53.27128896]] [0.14327936 0.15335458 0.06981551 0.02060609] 57.16363828166282
0.0 [[-52.84498159]] [0.14272826 0.15276455 0.06875638 0.02088436] 0.0
0.0 [[-52.84498236]] [0.14272827 0.15276455 0.06875638 0.02088436] 0.0
0.0 [[-52.84498232]] [0.14272826 0.15276456 0.06875638 0.02088436] 0.0
0.0 [[-52.84498324]] [0.14272826 0.15276455 0.06875639 0.02088436] 0.0
0.0 [[-52.84497537]] [0.14272826 0.15276455 0.06875638 0.02088437] 0.0
0.2438156202474571 [[-53.12803466]] [0.14309198 0.15315397 0.0694554  0.02070069] 24.38156202474571
0.2438178638990653 [[-53.12803542]] [0.14309199 0.15315397 0.0694554  0.02070069] 24.381786389906527
0.24381798314496955 [[-53.12803538]] [0.14309198 0.15315398 0.0694554  0.02070069] 24.381798314496955
0.24381812020442975 [[-53.12803621]] [0.1

0.0 [[-83.79978432]] [1.37769984e+01 1.29839785e+01 2.92250934e+01 6.22340895e-03] 0.0
0.0 [[-83.79978218]] [1.37769984e+01 1.29839785e+01 2.92250934e+01 6.22341895e-03] 0.0
0.0 [[-83.81404186]] [1.37689486e+01 1.29763970e+01 2.92078211e+01 6.13952112e-03] 0.0
0.0 [[-83.81404185]] [1.37689486e+01 1.29763970e+01 2.92078211e+01 6.13952112e-03] 0.0
0.0 [[-83.81404187]] [1.37689486e+01 1.29763970e+01 2.92078211e+01 6.13952112e-03] 0.0
0.0 [[-83.81404187]] [1.37689486e+01 1.29763970e+01 2.92078211e+01 6.13952112e-03] 0.0
0.0 [[-83.81404064]] [1.37689486e+01 1.29763970e+01 2.92078211e+01 6.13953112e-03] 0.0
0.0 [[-83.81593321]] [1.37673478e+01 1.29748894e+01 2.92043864e+01 6.12283940e-03] 0.0
0.0 [[-83.81593321]] [1.37673478e+01 1.29748894e+01 2.92043864e+01 6.12283940e-03] 0.0
0.0 [[-83.81593322]] [1.37673478e+01 1.29748894e+01 2.92043864e+01 6.12283940e-03] 0.0
0.0 [[-83.81593322]] [1.37673478e+01 1.29748894e+01 2.92043864e+01 6.12283940e-03] 0.0
0.0 [[-83.81593218]] [1.37673478e+01 1.2974

C:\Users\berkc\Miniconda3\envs\R\lib\site-packages\ipykernel_launcher.py:286: RuntimeWarning: covariance is not positive-semidefinite.


In [21]:
ss

array([[0.01430051, 0.03245551, 0.02846592, 0.05352703, 0.01394904,
        0.01048549, 0.02519831, 0.01877855, 0.03583528]])

In [22]:
np.mean(rmse)

0.02033540430000229